## CNN-based peak picking model

This CNN peak picking model is used for comparison with AutoMS. The theory from [PeakOnly](https://pubs.acs.org/doi/10.1021/acs.analchem.9b04811) software, which takes the advantage of the
intrinsic ability of CNN to classify peak with the profiles.

Here we use the same training data as AutoMS for equitable evaluation. The training data are from [Schulze's study](https://www.mdpi.com/2218-1989/10/4/162). The manually picked true peaks are treated as positive samples. Since the negative samples are far more than the positives, we applied MSPD for choosing the worst peaks as negatives. Moreover, we also kept the number of positives and the negatives balanced. The hyper-parameter and the training details are described with the source codes bellow. 

### Import packages

In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import optimizers
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from AutoMS.mspd_original import peaks_detection

### Data preparation

We transformed the training data in advance, where X is the matrix of ROIs and Y is the corresponding labels. Here we load the data directly.

In [2]:
x = np.load('data/X.npy')
y = np.load('data/Y.npy')

Then, scale the intensity to 0 - 1, and remove the rows with all 0.

In [3]:
for i in range(x.shape[0]):
    x[i,:] = x[i,:] / np.max(x[i,:])

sums = np.sum(x, axis = 1)
keep = np.where(sums > 0)[0]
x = x[keep, :]
y = y[keep]

C:\Users\DELL\AppData\Local\Temp\ipykernel_14588\1097894179.py:2: RuntimeWarning: invalid value encountered in true_divide
  x[i,:] = x[i,:] / np.max(x[i,:])


Split the positive and negative samples with the labels and see how many of them

In [4]:
pos = np.where(y == 1)[0]
x_true = x[pos,:]
print('Number of positives: {}'.format(len(x_true)))

neg = np.where(y == 0)[0]
x_false = x[neg,:]
print('Number of negatives: {}'.format(len(x_false)))

Number of positives: 99474
Number of negatives: 264116


Here, we use MSPD for detecting peaks in the negative samples. Only keep those samples failed to identified peaks with MSPD.

In [5]:
false = []
for i in range(x_false.shape[0]): 
    pks, sigs, snrs_ = peaks_detection(x_false[i,:], np.arange(1, 30), 0)
    criterion_1 = np.sum(x_false[i,:] == 0) < 5
    if not criterion_1:
        continue
    snrs_.append(0)
    criterion_2 = np.max(snrs_) <= 3
    if criterion_2:
        false.append(i)
    if len(false) == 99474:
        break
false = np.array(false)
x_false = x_false[false,:]

In [6]:
print('Number of negatives: {}'.format(len(x_false)))

Number of negatives: 68892


He, we combine the processed data.

In [7]:
x = np.vstack((x_true, x_false))
y = np.array([1] * len(x_true) + [0] * len(x_false))
y = np.vstack((y, 1-y)).T

### Define the CNN model

In [8]:
class CNN:
    def __init__(self, X, Y):
        X = np.expand_dims(X, -1)
        self.X = X
        self.Y = Y
        # train test split
        self.X_tr, self.X_ts, self.Y_tr, self.Y_ts = train_test_split(X, Y, test_size=0.1)
        
        inp = Input(shape=(X.shape[1:]))
        hid = inp
        
        # layer 1: filters: 32, kernel size: 2, activation: relu, padding: same, pooling size: 2
        hid = Conv1D(32, kernel_size=2, activation='relu')(hid)
        hid = MaxPooling1D(pool_size=2)(hid)
        
        # layer 2: filters: 16, kernel size: 2, activation: relu, padding: same, pooling size: 2   
        hid = Conv1D(16, kernel_size=2, activation='relu')(hid)
        hid = MaxPooling1D(pool_size=2)(hid)
        
        # layer 3: filters: 16, kernel size: 2, activation: relu, padding: same, pooling size: 2       
        hid = Conv1D(16, kernel_size=2, activation='relu')(hid)
        hid = MaxPooling1D(pool_size=2)(hid)
        
        # layer dense: nodes: 32, activation: relu
        hid = Flatten()(hid)
        hid = Dense(32, activation="relu")(hid)
        
        # output layer for classification
        prd = Dense(2, activation="softmax")(hid)
        
        # optimizer: adam, loss function: categorical crossentropy
        opt = optimizers.Adam(lr=0.001)
        model = Model(inp, prd)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
        self.model = model
    
    def train(self, epochs=5):
        self.model.fit(self.X_tr, self.Y_tr, validation_split= 0.1, epochs=epochs)
    
    def test(self):
        Y_pred = np.round(self.model.predict(self.X_ts))
        f1 = f1_score(self.Y_ts[:,0], Y_pred[:,0])
        precision = precision_score(self.Y_ts[:,0], Y_pred[:,0])
        recall = recall_score(self.Y_ts[:,0], Y_pred[:,0])
        accuracy = accuracy_score(self.Y_ts[:,0], Y_pred[:,0])
        return accuracy, precision, recall, f1
    
    def save(self, path):
        self.model.save(path)

### Training and evaluation

In [9]:
cnn = CNN(x, y)
cnn.train(epochs = 10)
accuracy, precision, recall, f1 = cnn.test()

print('accuracy: {}'.format(accuracy))
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('f1: {}'.format(f1))

Epoch 1/10


C:\Users\DELL\anaconda3\envs\py38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


4262/4262 [==============================] - 25s 5ms/step - loss: 0.2393 - acc: 0.9066 - val_loss: 0.2144 - val_acc: 0.9181
Epoch 2/10
4262/4262 [==============================] - 20s 5ms/step - loss: 0.2121 - acc: 0.9206 - val_loss: 0.2089 - val_acc: 0.9222
Epoch 3/10
4262/4262 [==============================] - 20s 5ms/step - loss: 0.2060 - acc: 0.9233 - val_loss: 0.2060 - val_acc: 0.9234
Epoch 4/10
4262/4262 [==============================] - 20s 5ms/step - loss: 0.2036 - acc: 0.9244 - val_loss: 0.1993 - val_acc: 0.9275
Epoch 5/10
4262/4262 [==============================] - 20s 5ms/step - loss: 0.2005 - acc: 0.9253 - val_loss: 0.1992 - val_acc: 0.9267
Epoch 6/10
4262/4262 [==============================] - 20s 5ms/step - loss: 0.1994 - acc: 0.9261 - val_loss: 0.2018 - val_acc: 0.9265
Epoch 7/10
4262/4262 [==============================] - 19s 5ms/step - loss: 0.1981 - acc: 0.9265 - val_loss: 0.1951 - val_acc: 0.9281
Epoch 8/10
4262/4262 [==============================] - 19s 5ms/st

In [10]:
# cnn.save('model/cnn.pkl')